# GOAL
The combined data will be parsed to produce the following data:

0. combined_data, but with title prepended to body text and `<p>` tags removed.
1. [ id | word_root (or ngram) | is_keyword ] columned data frame 
2. [ row_num (new id) | keyword | n_occurrences ] columned data frame
3. (2), but without taking the word root

(note that I am fairly new to Pandas)

In [5]:
import pandas as pd
#in case extra operations are necessary
import os
import numpy as np
#to remove html tags from questions
from bs4 import BeautifulSoup
#for alternate faster method for simple html structures
import re
#for summary statistics of word tags
from collections import Counter

In [2]:
data = pd.read_csv('combined_data.csv')

In [3]:
#also converts to lowercase
def remove_non_ascii(x):
    return (''.join([s for s in x if ord(s) < 128])).lower()

data['fulltext'] = data['title'].map(remove_non_ascii) + ' '

print 'initialized fulltext column' 

def process_question(x):
    #alternate, faster method:
    x = re.sub('</?(p|a href="[^"]*"|img src="[^"]*" alt="[^"]*"|table|h[1-9]|div|br|code|tr|th|ol|li|ul)/?>','',x)
    return remove_non_ascii(x)
    #soup = BeautifulSoup(x, 'lxml')
    #ps = soup.find_all('p')
    #text = ' '.join([p.text for p in ps])
    #return remove_non_ascii(text)

nrows = data.shape[0]

for row in xrange(nrows):
    if row % 1000 == 0:
        #normally does about 3,000 per minute
        #print 'row %d\r' % row 
        pass
    rowloc = data.iloc[row]
    rowloc['fulltext'] += process_question(rowloc['content'])
    
    

initialized fulltext column
row 0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


row 1000
row 2000
row 3000
row 4000
row 5000
row 6000
row 7000
row 8000
row 9000
row 10000
row 11000
row 12000
row 13000
row 14000
row 15000
row 16000
row 17000
row 18000
row 19000
row 20000
row 21000
row 22000
row 23000
row 24000
row 25000
row 26000
row 27000
row 28000
row 29000
row 30000
row 31000
row 32000
row 33000
row 34000
row 35000
row 36000
row 37000
row 38000
row 39000
row 40000
row 41000
row 42000
row 43000
row 44000
row 45000
row 46000
row 47000
row 48000
row 49000
row 50000
row 51000
row 52000
row 53000
row 54000
row 55000
row 56000
row 57000
row 58000
row 59000
row 60000
row 61000
row 62000
row 63000
row 64000
row 65000
row 66000
row 67000
row 68000
row 69000
row 70000
row 71000
row 72000
row 73000
row 74000
row 75000
row 76000
row 77000
row 78000
row 79000
row 80000
row 81000
row 82000
row 83000
row 84000
row 85000
row 86000


In [4]:
data.to_csv('data_00.csv', index=False)
data['fulltext'][0:20]

0     how do i install a new, non load bearing wall ...
1     what kind of caulk should i use around my bath...
2     is fiberglass mesh tape a good choice for dryw...
3     are there ways to determine if a wall is load ...
4     how do i safely replace a worn out electrical ...
5           how do i repair hairline cracks in stucco? 
6     where can i find a bathroom fan that will fit ...
7     how do i repair a small leak in a steam radiat...
8              how do i remove wallpaper from drywall? 
9     what do i need to do to add stools to my vinyl...
10    how can i hanging something on an exterior wal...
11        how can i fill a void in a concrete seawall? 
12    how to drain a baseboard radiator system for a...
13        should old knob and tube wiring be replaced? 
14    how do i find studs when stud finders won't wo...
15    what do i have to do to prepare my garage floo...
16    what is the difference between a framing hamme...
17    how do i remove dried paint from ceramic t

## Collect Keywords

Now I will gather all the keywords/phrases from each source

In [6]:
keyword_counters = {key:Counter() for key in ['diy.csv',
                                   'cooking.csv',
                                   'biology.csv', 
                                   'crypto.csv',
                                   'robotics.csv',
                                   'travel.csv']}
for i, row in data.iterrows():
    tags = row['tags']
    separated = tags.split(' ')
    source = row['source']
    for tag in separated:
        keyword_counters[source][tag] += 1

In [7]:
for key, counter in keyword_counters.iteritems():
    print 'source: %s | n_keywords: %d' % (key, len(counter))


source: diy.csv | n_keywords: 734
source: biology.csv | n_keywords: 678
source: robotics.csv | n_keywords: 231
source: travel.csv | n_keywords: 1645
source: cooking.csv | n_keywords: 736
source: crypto.csv | n_keywords: 392


and let's break down the compositions of the lengths of each of these keywords...

In [8]:
keyword_lengths = {key:Counter() for key in keyword_counters}
for key, counter in keyword_counters.iteritems():
    for keyword in counter:
        keylength = 1 + keyword.count('-')
        keyword_lengths[key][keylength] += 1
        
for key, counter in keyword_lengths.iteritems():
    print 'source: %s' % key
    for keylength in counter:
        print 'count of length %d: %d' % (keylength, counter[keylength])
        

source: diy.csv
count of length 1: 532
count of length 2: 184
count of length 3: 18
source: biology.csv
count of length 1: 474
count of length 2: 189
count of length 3: 15
source: robotics.csv
count of length 1: 163
count of length 2: 65
count of length 3: 3
source: travel.csv
count of length 1: 1053
count of length 2: 507
count of length 3: 75
count of length 4: 10
source: cooking.csv
count of length 1: 551
count of length 2: 177
count of length 3: 7
count of length 4: 1
source: crypto.csv
count of length 1: 187
count of length 2: 163
count of length 3: 40
count of length 4: 1
count of length 5: 1


now lets get raw counts of total occurrences by length; it looks like we will only need to focus on length-1 and length-2 ngrams, as lengths of 3 are fairly rare (<10% for each category except cryptography, <5% for most categories). Since the highest accuracy achieved realistically is around 30%, this isn't a huge loss 

there may be obvious keywords that are length-3 that could be found using traditional methods, so that could always be used in case words like "large hadron collider" (possibly shortened to LHC) are used. actually, acronyms for larger phrases are probably more likely

In [9]:
keyword_length_counts = {key:Counter() for key in keyword_counters.keys()}

for key, counter in keyword_counters.iteritems():
    for keyword, count in counter.iteritems():
        keyword_length = 1 + keyword.count('-')
        keyword_length_counts[key][keyword_length] += count
        
for key, counter in keyword_length_counts.iteritems():
    print 'source: %s' % key
    for keylength, count in counter.iteritems():
        print 'combined total of length %d: %d' % (keylength, count)
        

source: diy.csv
combined total of length 1: 49725
combined total of length 2: 8644
combined total of length 3: 760
source: biology.csv
combined total of length 1: 23045
combined total of length 2: 9846
combined total of length 3: 238
source: robotics.csv
combined total of length 1: 4642
combined total of length 2: 1870
combined total of length 3: 8
source: travel.csv
combined total of length 1: 45388
combined total of length 2: 15675
combined total of length 3: 3848
combined total of length 4: 423
source: cooking.csv
combined total of length 1: 28042
combined total of length 2: 7337
combined total of length 3: 116
combined total of length 4: 47
source: crypto.csv
combined total of length 1: 13412
combined total of length 2: 9624
combined total of length 3: 2378
combined total of length 4: 41
combined total of length 5: 29


This reinforces the idea that focusing on lengths 1 and 2 of ngrams is most productive at this time. length-3 may be visited later, and lengths of 4 and 5 are far too rare to worry about

Now let's save these keywords as pandas frames. 

In [24]:
def addcol(varname, value, x):
    x[varname] = value
    return x

keyword_counter_df = pd.concat([addcol('source',source,
                        pd.DataFrame.from_dict(counter, orient='index'))  
                        for source, counter in keyword_counters.iteritems()])

In [25]:
keyword_counter_df.index.name = 'keyword'
keyword_counter_df.reset_index(inplace=True)


In [26]:
keyword_counter_df.rename(columns={0:'count'}, inplace=True)
keyword_counter_df

,keyword,count,source
0,pocket-door,14,diy.csv
1,splicing,11,diy.csv
2,irrigation,39,diy.csv
3,product-review,7,diy.csv
4,chain,7,diy.csv
5,glass-top-range,6,diy.csv
6,forced-air,13,diy.csv
7,lubrication,35,diy.csv
8,lally,1,diy.csv
9,hanging,86,diy.csv


Lastly, sort the entries by group then count descending, and create an index for words within each group. Calculate cumulative percentage of counts within groups for a good idea of the distribution of terms.

In [51]:
sorted_df = keyword_counter_df.sort_values(by=['source','count'], 
                                        ascending=[True, False])

sorted_df['group_idx'] = sorted_df.groupby(['source']).cumcount()
sorted_df['cumulative_proportion'] = sorted_df.groupby('source')['count']\
    .apply(lambda x: (x.cumsum() / x.sum() * 100).round(2) )
    
sorted_df.reset_index(inplace=True)

In [52]:
sorted_df

,index,keyword,count,source,group_idx,cumulative_proportion
0,853,human-biology,1448,biology.csv,0,4.37
1,1300,genetics,1229,biology.csv,1,8.08
2,1089,evolution,1159,biology.csv,2,11.58
3,1378,biochemistry,984,biology.csv,3,14.55
4,963,molecular-biology,863,biology.csv,4,17.15
5,1232,cell-biology,759,biology.csv,5,19.45
6,876,bioinformatics,663,biology.csv,6,21.45
7,831,dna,637,biology.csv,7,23.37
8,1393,neuroscience,614,biology.csv,8,25.22
9,894,botany,565,biology.csv,9,26.93


In [54]:
sorted_df.to_csv('sorted_keyword_counts.csv', index=False)

Next time, we're going to look at how frequently these phrases appear in their relevant text. After that, we will construct a network graph in Gephi to look at clustering, and after that, we should have a good enough idea of how to go about designing and creating the models.